<a href="https://colab.research.google.com/github/prithvirao/NLP-Project-1/blob/main/NLP_project_1_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers
!pip install sentencepiece
!pip install tweepy==3.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip uninstall nltk
!pip install -U nltk

Found existing installation: nltk 3.8.1
Uninstalling nltk-3.8.1:
  Would remove:
    /usr/local/bin/nltk
    /usr/local/lib/python3.8/dist-packages/nltk-3.8.1.dist-info/*
    /usr/local/lib/python3.8/dist-packages/nltk/*
Proceed (Y/n)? y
  Successfully uninstalled nltk-3.8.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)


In [5]:
import tweepy
import re
import nltk
import math
import spacy
from nltk.corpus import stopwords
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datetime import datetime, timedelta

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [6]:
# set up Twitter API credentials
consumer_key = "skakU2irbEBd97RvhpBsW0sXL"
consumer_secret = "Bgb8TuOlhCB0nRBpyJCMtKOLkvhuSW7D0V1EoJzf7p7McFwknf"
access_token = "880363396016869376-fFRlVcD2l7sMDmpGZw3yZelKxijcwEP"
access_token_secret = "l0QpT9WuWXHCV4on1C20irIzXYrtxIoHoJXHtSh7vxG4j"

In [7]:
# set up API authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [8]:
# set up API client
api = tweepy.API(auth)

In [9]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def preprocess_text(text):
    text = re.sub(r'http\S+', '', text)  # remove urls
    text = re.sub(r'\#\w+\rt', '', text)  # remove hashtags
    text = text.lower()  # convert to lowercase
    text = ' '.join([word for word in text.split() if word not in stop_words])  # remove stop words
    return text

In [11]:
# Extract relevant tweets based on keywords and dates
start_date = datetime.now() - timedelta(days=5)
end_date = datetime.now()

In [13]:
# Collect tweets for each day
for i in range((end_date - start_date).days):
    day = start_date + timedelta(days=i)
    date_str = day.strftime("%Y-%m-%d")
    
    # Extract relevant tweets based on keywords
    keywords = ['earthquake']
    tweets = []
    primary_locations={'testing':0}
    location_summaries={}
    for keyword in keywords:
        for tweet in tweepy.Cursor(api.search, q=keyword, lang='en', tweet_mode='extended', until=date_str).items(100):
            text = tweet.full_text
            preprocessed_text = preprocess_text(text)
            tweets.append(preprocessed_text)
            
            # Extract location entities from tweet text
            doc = nlp(preprocessed_text)
            locations = [ent.text for ent in doc.ents if ent.label_ == 'GPE']
            
            if len(locations)>1:
              locations=[locations[0]]

            for location in locations:
              if location not in primary_locations:
                primary_locations[location]=0
              else:
                primary_locations[location]=primary_locations[location]+1

              if location not in location_summaries:
                location_summaries[location] = []
              location_summaries[location].append(preprocessed_text)
            
    print(primary_locations)
    #print(primary_locations.values())
    max_value = max(primary_locations.values())
    #print(max_value) 

    threshold=math.floor(math.sqrt(max_value))

    # Rank and summarize the relevant tweets using a machine learning-based model (T5)
    tokenizer = T5Tokenizer.from_pretrained('t5-base')
    model = T5ForConditionalGeneration.from_pretrained('t5-base')
    
    #prev_sum="";
    print()
    print()
    print("#"*20)
    print(f"Summary for {date_str}:")
    print("#"*20)
    for location, location_tweets in location_summaries.items():
        if primary_locations[location]>=threshold:
          input_text = ' '.join(location_tweets)
          input_ids = tokenizer.encode(input_text, return_tensors='pt')
          summary_ids = model.generate(input_ids, num_beams=4, length_penalty=2.0, max_length=100, min_length=10, no_repeat_ngram_size=2)
          summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
          #if prev_sum != summary:
          print(f"Summaries for {location}:")
          # Output summary for each location
          print(summary)
          print()
          #prev_sum=summary


{'testing': 0, 'saudi arabia': 0, 'syria': 15, 'canada': 1, 'turkey': 2, 'israel': 5, 'hungary': 0, 'north korea': 0, 'west island': 0, 'usgs': 0, 'china': 0, 'blenheim': 3, 'hong kong': 0, 'new zealand': 0}


Token indices sequence length is longer than the specified maximum sequence length for this model (520 > 512). Running this sequence through the model will result in indexing errors




####################
Summary for 2023-02-22:
####################
Summaries for syria:
rt @chuckcallesto: massive 300 km long crack earth's crust revealed turkey syria earthquake.. lokanta mediterranean grill &amp; bar donate sales made wednesday help earthquake victims turk...

Summaries for israel:
rt @qudsnen: turns israel contingent sent ostensibly 'help' victims turkey/syria earthquake loo... @brendanbeirne @jasonlemon maybe could write a shambles about the bombing of damascus, another famous serial killer state bombs northern part. earthquake hit, states murdered people. disgusting realities change? think media much hides fact

Summaries for blenheim:
m4.1 quake causing weak shaking near blenheim rt @geonet: m3.2 tremor resulting in weak shake near the city of st. johns-lee qc - earthquakes in europe and the pacific neo-nazis equanes in the west of the united states aquakes in germany and u.s. earthquake

{'testing': 0, 'turkey': 3, 'syria': 9, 'iran': 0, 'usgs': 1, 'united sta

Token indices sequence length is longer than the specified maximum sequence length for this model (641 > 512). Running this sequence through the model will result in indexing errors




####################
Summary for 2023-02-24:
####################
Summaries for syria:
rt @abc3340: death toll massive earthquake hit parts turkey syria feb. 6 continues rise bodies are... click support urgent! turkish earthquake relief organized danita cinquemani #gofundme please give love help heal lost loved ones like poor man.

Summaries for israel:
rt @hadinasrallah: one week israel bombed syria's capital.. id bombings in lebanese village massacred palestinians wes...

{'testing': 0, 'india': 0, 'israel': 0, 'syria': 12, 'usgs': 2, 'mentasta lake': 0, 'united states': 0, 'australia': 0, 'canada': 0, 'iraq': 7, 'japan': 1, 'turkey': 2, '@raahiagbah': 0, 'egypt': 0}


Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors




####################
Summary for 2023-02-25:
####################
Summaries for syria:
rt @warmonitors: #breaking total earthquake death toll surpasses 50,000 turkey syria. ivan_8848: children protest western sanctions dozens parents children took streets damascus, dunya needs surgery 02/24/2023 choose link latest update. consider donating elevate prize winner'sana-mustafa-1' venmo: https://www.a

Summaries for iraq:
#earthquake (#) possibly felt 21 sec ago #iraq. colored dots represent local shaking &amp; damage level reported eyewitnesses.

{'testing': 0, 'hungary': 0, 'sacramento': 0, 'japan': 5, 'turkey': 0, 'syria': 7, 'konya': 0, 'canada': 0, 'rt @gmrpetricca': 0, 'pakistan': 0, 'britain': 0, 'united kingdom': 0, 'lebanon': 0, 'california': 0, 'britain province': 0}


####################
Summary for 2023-02-26:
####################
Summaries for japan:
rsoe edis: earthquake 6.1 magnitude hits northern japan, tsunami warns ua. earthquake struck hokkaido on tuesday, usgs read:  